# Gold Layer – Race Results

## Objective
This notebook creates the `gold.race_results` table by enriching race result data from the Silver layer.  
We join **session_results** with **circuit_info** to build a consolidated race results dataset that includes:
- Driver and team details  
- Circuit and event metadata  
- Derived race metrics  

## Steps
1. Load Silver layer tables: `session_results`, `circuit_info`.  
2. Join on `session_key` and `year+round` where needed.  
3. Select and enrich columns: driver, team, circuit, event info.  
4. Compute derived columns such as:
   - `race_duration` (end-to-start time difference)  
   - `is_podium` flag (position <= 3)  
   - `is_fastest` flag for fastest lap in race  
5. Remove duplicates and handle nulls.  
6. Write the transformed data into the Gold layer as a Delta table `gold.race_results`.  
7. Optimize with ZORDER on `(year, round)` for query performance.  


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 1. Load Silver tables
session_results = spark.table("silver.session_results")
circuit_info = spark.table("silver.circuit_info")

# 2. Join session_results with circuit_info
race_results_df = (
    session_results.alias("sr")
    .join(
        circuit_info.alias("ci"),
        on=["session_key", "year", "round"],
        how="left"
    )
)

# 3. Select relevant columns & enrich
race_results_df = (
    race_results_df.select(
        "sr.DriverId", "sr.DriverNumber", "sr.FirstName", "sr.LastName", "sr.FullName",
        "sr.Abbreviation", "sr.TeamName", "sr.TeamId", "sr.TeamColor",
        "sr.Position", "sr.GridPosition", "sr.Points", "sr.Status",
        "sr.Laps", "sr.Time",
        "ci.circuit_name", "ci.circuit_country", "ci.location", "ci.distance",
        "ci.event_name", "ci.session_name", "ci.year", "ci.round", "ci.session_key"
    )
    .withColumn("is_podium", F.when(F.col("Position") <= 3, F.lit(True)).otherwise(F.lit(False)))
)

# 4. Compute fastest lap flag (window by session_key)
w = Window.partitionBy("session_key").orderBy(F.col("Time").asc_nulls_last())
race_results_df = race_results_df.withColumn("row_num", F.row_number().over(w))
race_results_df = race_results_df.withColumn("is_fastest", F.when(F.col("row_num") == 1, True).otherwise(False))
race_results_df = race_results_df.drop("row_num")

# 5. Remove duplicates
race_results_df = race_results_df.dropDuplicates()

# 6. Write to Gold layer
(
    race_results_df.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable("gold.race_results")
)

# 7. Optimize table
spark.sql("OPTIMIZE gold.race_results ZORDER BY (year, round)")